# Qdrant 101

![qdrant](https://qdrant.tech/images/logo_with_text.png)

Vector databases are a relatively new way for interacting with abstract data representations derived from opaque machine learning models such as deep learning architectures. These representations are often called vectors or embeddings and they are a compressed version of the data used to train a machine learning model to accomplish a task like sentiment analysis, speech recognition, object detection, and many others.

These new databases shine in many applications like [semantic search](https://en.wikipedia.org/wiki/Semantic_search) and [recommendation systems](https://en.wikipedia.org/wiki/Recommender_system), and in this tutorial, we'll learn about how to get started with one of the most popular and fastest growing vector databases in the market, [Qdrant](qdrant.tech).

## Table of Contents

1. Learning Outcomes
2. What is Qdrant?
    - What are Vector Databases?
    - Why do We Need Vector Databases??
    - Overview of Qdrant's Architecture    
    - Installation
3. Getting Started
    - Adding Points
    - Payload
    - Search
4. Recommendations
5. Conclusion
6. Resources

## 1. Learning Outcomes

By the end of this tutorial, you will be able
- Describe what vector databases are and what are they used for.
- Create, update, and query collections of vectors using Qdrant.
- Conduct semantic search based on new data.
- Understand the mechanics the recommendation engine of Qdrant.

## 2. What is Qdrant?

[Qdrant](qdrant.tech) "is a vector similarity search engine that provides a production-ready service with a convenient API to store, search, and manage points (i.e. vectors) with an additional payload." You can think of the payloads as additional pieces of information that can help you hone in on your search while also returning useful information to your users (we'll talk more about the payload functionality in a bit).

You can get started using Qdrant with the Python `qdrant-client`, by pulling the latest docker image of `qdrant` and connecting to it locally, or by trying out Qdrant's Cloud free tier option until you are ready to make the full switch.

With that out of the way, let's talk about what are vector databases.

### 2.1 What Are Vector Databases?

![dbs](../images/databases.png)

Vector databases are a type of database designed to store and query high-dimensional vectors efficiently. In traditional [OLTP](https://www.ibm.com/topics/oltp) and [OLAP](https://www.ibm.com/topics/olap) databases (as seen in the image above), data is organized in rows and columns, and queries are performed based on the values in those columns. However, in certain applications including image recognition, natural language processing, and recommendation systems, data is often represented as vectors in a high-dimensional space, and these vectors, plus an id and a payload, are the elements we store in a vector database like Qdrant.

A vector in this context is a mathematical representation of an object or data point, where each element of the vector corresponds to a specific feature or attribute of the object. For example, in an image recognition system, a vector could represent an image, with each element of the vector representing a pixel value or a descriptor/characteristic of that pixel.

Vector databases are optimized for **storing** and **querying** these high-dimensional vectors efficiently, often using specialized data structures and indexing techniques such as Hierarchical Navigable Small World (HNSW) -- which is used to implement Approximate Nearest Neighbors -- and Product Quantization, among others. These databases enable fast similarity and semantic search while allowing users to find vectors that are the closest to a given query vector based on some distance metric. The most commonly used distance metrics are Euclidean Distance, Cosine Similarity, and Dot Product.

Now that we know what vector databases are, and how they are structurally different than other databases, let's go over why they are important.

### 2.2 Why do we need Vector Databases?

Vector databases play a crucial role in various applications that require similarity search, such as recommendation systems, content-based image retrieval, and personalized search. By taking advantage of their efficient indexing and searching techniques, vector databases enable faster and more accurate retrieval of similar vectors, which helps advance data analysis and decision-making.

In addition, other benefits of using vector databases include:
1. Efficient storage and indexing of high-dimensional data.
3. Ability to handle large-scale datasets with billions of data points.
4. Support for real-time analytics and queries.
5. Ability to handle vectors derived from complex data types such as images, videos, and natural language text.
6. Improved performance and reduced latency in machine learning and AI applications.
7. Reduced development and deployment time and cost compared to building a custom solution.

Keep in mind that the specific benefits of using a vector database may vary depending on the use case of your organization and the features of the database you ultimately choose.

Let's now evaluate, at a high-level, the way Qdrant is architected.

### 2.3 Overview of Qdrant's Architecture (High-Level)

![qdrant](../images/qdrant_overview_high_level.png)

The diagram above represents a high-level overview of some of the main components of Qdrant. Here are the terminologies you should get familiar with.

- [Collections](https://qdrant.tech/documentation/collections/): A collection is a named set of points (vectors with a payload) among which you can search. Vectors within the same collection can have different dimensionalities and be compared by a single metric.
- [Distance Metrics](https://en.wikipedia.org/wiki/Metric_space): These are used to measure similarities among vectors and they must be selected at the same time you are creating a collection. The choice of metric depends on the way the vectors were obtained and, in particular, on the neural network that will be used to encode new queries.
- [Points](https://qdrant.tech/documentation/points/): The points are the central entity that Qdrant operates with and they consist of a vector and an optional id and payload.
    - id: a unique identifier for your vectors.
    - Vector: a high-dimensional representation of data, for example, an image, a sound, a document, a video, etc.
    - [Payload](https://qdrant.tech/documentation/payload/): A payload is a JSON object with additional data you can add to a vector.
- [Storage](https://qdrant.tech/documentation/storage/): Qdrant can use one of two options for storage, **In-memory** storage (Stores all vectors in RAM, has the highest speed since disk access is required only for persistence), or **Memmap** storage, (creates a virtual address space associated with the file on disk).
- Clients: the programming languages you can use to connect to Qdrant.

### 2.4 How do we get started?

The open source version of Qdrant is available as a docker image and it can be pulled and run from any machine with docker installed. If you don't have Docker installed in your PC you can follow the instructions in the official documentation [here](https://docs.docker.com/get-docker/). After that, open your terminal start by downloading the image with the following command.

```sh
docker pull qdrant/qdrant
```

Next, initialize Qdrant with the following command, and you should be good to go.

```sh
docker run -p 6333:6333 \
    -v $(pwd)/qdrant_storage:/qdrant/storage \
    qdrant/qdrant
```

You should see something similar to the following image.

![dockerqdrant](../images/docker_qdrant.png)

If you experience any issues during the start process, please let us know in our [discord channel here](https://qdrant.to/discord). We are always available and happy to help.

Now that you have Qdrant up and running, your next step is to pick a client to connect to it. We'll be using Python as it has the most mature data tools' ecosystem out there. So, let's start setting up our dev environment and getting the libraries we'll be using today.

```sh
# with mamba or conda
mamba env create -n my_env python=3.10
mamba activate my_env

# or with virtualenv
python -m venv venv
source venv/bin/activate

# install packages
pip install qdrant-client pandas numpy faker
```

After your have your environment ready, let's get started using Qdrant.

**Note:** At the time of writing, Qdrant supports Rust, GO, Python and TypeScript. We expect other programming languages to be added in the future.

## 3. Getting Started

The two modules we'll use the most are the `QdrantClient` and the `models` one. The former allows us to connect to Qdrant or it allows us to run an in-memory database by switching the parameter `location=` to `":memory:"` (this is a great feature for testing in a CI/CD pipeline). We'll start by instantiating our client using `host="localhost"` and `port=6333` (as it is the default port we used earlier with docker). You can also follow along with the `location=":memory:"` option commented out below.

In [1]:
from qdrant_client import QdrantClient
from qdrant_client.http import models
from qdrant_client.http.models import CollectionStatus

In [2]:
client = QdrantClient(host="localhost", port=6333)
client

In [ ]:
# client = QdrantClient(location=":memory:")
# client

In OLTP and OLAP databases we call specific bundles of rows and columns **Tables**, but in vector databases, the rows are known as vectors, the columns are known as dimensions, and the combination of the two (plus some metadata) as **collections**.

In the same way in which we can create many tables in an OLTP or an OLAP database, we can create many collections in a vector database like Qdrant using one of its clients. The key difference to note is that when we create a collection in Qdrant, we need to specify the width of the collection (i.e. the length of the vector or amount of dimensions) beforehand with the parameter `size=...`, as well as the distance metric with the parameter `distance=...` (which can be changed later on).

The distances currently supported by Qdrant are:
- [**Cosine Similarity**](https://en.wikipedia.org/wiki/Cosine_similarity) - Cosine similarity is a way to measure how similar two things are. Think of it like a ruler that tells you how far apart two points are, but instead of measuring distance, it measures how similar two things are. It's often used with text to compare how similar two documents or sentences are to each other. The output of the cosine similarity ranges from 0 to 1, where 0 means the two things are completely dissimilar, and 1 means the two things are exactly the same. It's a straightforward and effective way to compare two things!
- [**Dot Product**](https://en.wikipedia.org/wiki/Dot_product) - The dot product similarity metric is another way of measuring how similar two things are, like cosine similarity. It's often used in machine learning and data science when working with numbers. The dot product similarity is calculated by multiplying the values in two sets of numbers, and then adding up those products. The higher the sum, the more similar the two sets of numbers are. So, it's like a scale that tells you how closely two sets of numbers match each other.
- [**Euclidean Distance**](https://en.wikipedia.org/wiki/Euclidean_distance) - Euclidean distance is a way to measure the distance between two points in space, similar to how we measure the distance between two places on a map. It's calculated by finding the square root of the sum of the squared differences between the two points' coordinates. This distance metric is commonly used in machine learning to measure how similar or dissimilar two data points are or, in other words, to understand how far apart they are.

Let's create our first collection and have the vectors be of size 100 with a distance set to **Cosine Similarity**. Please note that, at the time of writing, Qdrant only supports cosine similarity, dot product and euclidean distance for its distance metrics.

In [3]:
my_collection = "first_collection"

first_collection = client.recreate_collection(
    collection_name=my_collection,
    vectors_config=models.VectorParams(size=100, distance=models.Distance.COSINE)
)
print(first_collection)

True


We can extract information related to the health of our collection by getting the collection. In addition, we can use this information for testing purposes, which can be very beneficial while in development mode.

In [4]:
collection_info = client.get_collection(collection_name=my_collection)
list(collection_info)

[('status', <CollectionStatus.GREEN: 'green'>),
 ('optimizer_status', <OptimizersStatusOneOf.OK: 'ok'>),
 ('vectors_count', 0),
 ('indexed_vectors_count', 0),
 ('points_count', 0),
 ('segments_count', 8),
 ('config',
  CollectionConfig(params=CollectionParams(vectors=VectorParams(size=100, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None), shard_number=1, replication_factor=1, write_consistency_factor=1, on_disk_payload=True), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=1), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=None)),
 ('payload_schema', {})]

In [5]:
assert collection_info.status == CollectionStatus.GREEN
assert collection_info.vectors_count == 0

There's a couple of things to notice from what we have done so far.
- The first is that when we initiated our docker image, we created a local directory called, `qdrant_storage`, and this is where all of our collections, plus their metadata, will be saved at. You can have a look at that directory in a *nix system with `tree qdrant_storage -L 2`, and something similar to the following output should come up for you.
    ```bash
    qdrant_storage
    ├── aliases
    │   └── data.json
    ├── collections
    │   └── my_first_collection
    └── raft_state
    ```
- The second is that we used `client.recreate_collection` and this command, as the name implies, can be used more than once to create new collections with or without the same name, so be careful no to recreate a collection that you did not intend to recreate. To create a brand new collection that cannot be recreated again, we would use `client.create_collection` instead.
- Our collection will hold vectors of 100 dimensions and the distance metric has been set to Cosine Similarity.

Now that we know how to create collections, let's create a bit of fake data and add some vectors to it.

### 3.1 Adding Points

The points are the central entity Qdrant operates with, and these contain records consisting of a vector, an optional `id` and an optional `payload` (which we'll talk more about in the next section).

The optional id can be represented by [unsigned integers](https://en.wikipedia.org/wiki/Integer_(computer_science)) or [UUID(https://en.wikipedia.org/wiki/Universally_unique_identifier)]s but, for our use case, we will use a straightforward range of numbers.

Let's us [NumPy](https://numpy.org/) to create a matrix of fake data containing 1,000 vectors and 100 dimensions and represent the values as `float64` numbers between -1 and 1. For simplicity, let's imagine that each of these vectors represents one of our favorite songs, and that each columns represents a unique characteristic of the artists/bands we love, for example, the tempo, the beats, the pitch of the voice of the singer(s), etc.

In [6]:
import numpy as np

In [7]:
data = np.random.uniform(low=-1.0, high=1.0, size=(1_000, 100))
type(data[0, 0]), data[:2, :20]

(numpy.float64,
 array([[-0.25490239, -0.69711475, -0.91160495,  0.4286922 ,  0.30737574,
          0.24289566,  0.29450399, -0.54190091, -0.77936913,  0.26133774,
         -0.91981819,  0.74629397,  0.33890408, -0.87023841,  0.97318671,
         -0.56333498,  0.21672956,  0.18859776, -0.7568224 , -0.14425111],
        [-0.58496535,  0.95815902, -0.47002311,  0.92539159, -0.9406764 ,
          0.02406786, -0.73990739, -0.36636866, -0.91299784,  0.59581209,
          0.89639446,  0.06784529, -0.83216908,  0.01621759, -0.3824888 ,
         -0.65195109, -0.32396782, -0.88301928, -0.2408039 ,  0.14688678]]))

Let's now create an index for our vectors.

In [8]:
index = list(range(len(data)))
index[-10:]

[990, 991, 992, 993, 994, 995, 996, 997, 998, 999]

Once the collection has been created, we can fill it in with the command `client.upsert()`. We'll need the collection's name and the appropriate uploading process from our `models` module, in this case, [`Batch`](https://qdrant.tech/documentation/points/#upload-points).

One thing to note is that Qdrant can only take in native Python iterables like lists and tuples. This is why you'll notice the `.tolist()` method attached to our numpy matrix,`data`, below.

In [9]:
client.upsert(
    collection_name=my_collection,
    points=models.Batch(
        ids=index,
        vectors=data.tolist()
    )
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

We can retrieve specific points based on their ID (for example, artist X with ID 1000) and get some additional information from that result.

In [10]:
client.retrieve(
    collection_name=my_collection,
    ids=[100],
    with_vectors=True # the default is False
)

[Record(id=100, payload={}, vector=[0.06537885, 0.08377354, -0.1271498, 0.11201218, 0.14291202, 0.108191885, -0.050187122, -0.049878836, -0.08581385, 0.10416953, -0.10143669, -0.03420135, 0.037018213, -0.020675888, 0.063463606, -0.098550715, 0.062296778, -0.06844608, 0.080228, -0.0048112106, 0.15913717, 0.005952451, 0.09631151, -0.1493667, -0.0152216265, -0.13522221, 0.123638205, -0.010182614, -0.05317723, 0.1358844, 0.08274643, -0.091148265, 0.1366898, 0.11368535, -0.046615336, 0.17010127, 0.113197796, -0.0016135894, -0.04244345, 0.15406793, -0.104462065, 0.13566178, -0.10643328, 0.055025216, 0.11442678, 0.021121271, 0.16848494, -0.16146518, -0.11261275, -0.13372509, 0.12982503, -0.04216307, -0.02294014, 0.011974023, 0.1667254, -0.12406298, -0.089919746, 0.10431936, -0.1574673, -0.06838789, -0.12509286, 0.0077518513, -0.10443644, -0.1435852, -0.0972059, -0.008827728, 0.06270219, -0.054325346, -0.08306797, -0.02075181, 0.06555662, -0.09629928, -0.039846133, -0.1275561, -0.16981077, 0.0

We can also update our collection one point at a time, for example, as new data comes in.

In [11]:
def create_song():
    return np.random.uniform(low=-1.0, high=1.0, size=100).tolist()

In [12]:
client.upsert(
    collection_name=my_collection,
    points=[
        models.PointStruct(
            id=1000,
            vector=create_song(),
        )
    ]
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

We can also delete it in a straightforward fashion.

In [13]:
# this will show the amount of vectors BEFORE deleting the one we just created
client.count(
    collection_name=my_collection, 
    exact=True,
) 

CountResult(count=1001)

In [14]:
client.delete(
    collection_name=my_collection,
    points_selector=models.PointIdsList(
        points=[1000],
    ),
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [15]:
# this will show the amount of vectors AFTER deleting them
client.count(
    collection_name=my_collection, 
    exact=True,
)

CountResult(count=1000)

### 3.2 Payloads

Qdrant has incredible features on top of speed and reliability, and one of its most useful ones is without a doubt the ability to store additional information alongside the vectors. In Qdrant's terminology, this information is considered a payload and it is represented as JSON objects. With these payloads, not only can you get information back when you search in the database, but you can also filter your search by the parameters in the payload, and we'll see how in a second.

Imagine the fake vectors we created actually represented a song. If we were building a semantic search system for songs then, naturally, the things we would want to get back would be the song itself (or an URL to it), the artist, maybe the genre, and so on.

What we'll do here is to take advantage of a Python package call `faker` and create a bit of information to add to our payload and see how this functionality works.

In [16]:
from faker import Faker

In [17]:
fake_something = Faker()
fake_something.name()

'Stephanie Baxter'

For each vector, we'll create list of dictionaries containing the artist, the song, a url to the song, the year in which it was released, and the country where it originated from.

In [18]:
payload = []

for i in range(len(data)):
    payload.append(
        {
            "artist":   fake_something.name(),
            "song":     " ".join(fake_something.words()),
            "url_song": fake_something.url(),
            "year":     fake_something.year(),
            "country":  fake_something.country()
        }
    )

payload[:3]

[{'artist': 'Brandy White',
  'song': 'education as something',
  'url_song': 'https://nolan.info/',
  'year': '2010',
  'country': 'Zambia'},
 {'artist': 'Alexandra Moore',
  'song': 'task minute hot',
  'url_song': 'http://www.murray.com/',
  'year': '1995',
  'country': 'Spain'},
 {'artist': 'Kelly Smith',
  'song': 'decade on task',
  'url_song': 'https://delgado-cooper.com/',
  'year': '1977',
  'country': 'Japan'}]

We can upsert our Points (ids, data, and payload), with the same `client.upsert()` method we used earlier, and we can retrieve any one song with the `client.retrieve()` method.

In [19]:
client.upsert(
    collection_name=my_collection,
    points=models.Batch(
        ids=index,
        vectors=data.tolist(),
        payloads=payload
    )
)

UpdateResult(operation_id=3, status=<UpdateStatus.COMPLETED: 'completed'>)

In [20]:
resutls = client.retrieve(
    collection_name=my_collection,
    ids=[10, 50, 100, 500],
    with_vectors=False
)

type(resutls), resutls

(list,
 [Record(id=50, payload={'artist': 'Kristen Diaz', 'country': 'Sweden', 'song': 'understand how radio', 'url_song': 'https://james.biz/', 'year': '2010'}, vector=None),
  Record(id=10, payload={'artist': 'David Bowen', 'country': 'Papua New Guinea', 'song': 'floor mother value', 'url_song': 'http://www.webb.com/', 'year': '2023'}, vector=None),
  Record(id=100, payload={'artist': 'Tammy Solis', 'country': 'Estonia', 'song': 'audience third study', 'url_song': 'http://www.simmons-erickson.com/', 'year': '2001'}, vector=None),
  Record(id=500, payload={'artist': 'Karen Wade', 'country': 'Italy', 'song': 'city experience guess', 'url_song': 'http://www.wolf.biz/', 'year': '1979'}, vector=None)])

We got back a list with records and each element inside a record can be accessed as an attribute, e.g. `.payload` or `.id`.

In [21]:
resutls[0].payload

{'artist': 'Kristen Diaz',
 'country': 'Sweden',
 'song': 'understand how radio',
 'url_song': 'https://james.biz/',
 'year': '2010'}

In [22]:
resutls[0].id

50

Now that you know a little bit about the payload functionality of Qdrant, let's use it to search.

### 3.3 Search

Now that we have our vectors with an ID and a payload, we can explore a few of ways in which we can search for content when, in our use case, new music gets selected. Let's check it out.

Say, for example, that a new song (like ["living la vida loca"](https://www.youtube.com/watch?v=p47fEXGabaY&ab_channel=RickyMartinVEVO) by Ricky Martin) comes in and our model immediately transforms it into a vector. Since we don't want a large amount of values back, let's limit the search to 10 points.

In [23]:
living_la_vida_loca = create_song()

In [24]:
client.search(
    collection_name=my_collection,
    query_vector=living_la_vida_loca,
    limit=3
)

[ScoredPoint(id=671, version=3, score=0.31004974, payload={'artist': 'Lindsey English', 'country': 'Comoros', 'song': 'step project several', 'url_song': 'https://www.vega-turner.org/', 'year': '1980'}, vector=None),
 ScoredPoint(id=209, version=3, score=0.29157263, payload={'artist': 'Sarah Williams', 'country': 'Denmark', 'song': 'test chance anyone', 'url_song': 'http://www.campbell.com/', 'year': '1987'}, vector=None),
 ScoredPoint(id=809, version=3, score=0.28871408, payload={'artist': 'Kristine Hall', 'country': 'Namibia', 'song': 'room bank world', 'url_song': 'http://www.buchanan.com/', 'year': '1992'}, vector=None)]

Now imagine that we only want Australian songs recommended to us. For this, we can filter the query with a payload.

In [25]:
aussie_songs = models.Filter(
    must=[models.FieldCondition(key="country", match=models.MatchValue(value="Australia"))]
)
type(aussie_songs)

qdrant_client.http.models.models.Filter

In [26]:
client.search(
    collection_name=my_collection,
    query_vector=living_la_vida_loca,
    query_filter=aussie_songs,
    limit=2
)

[ScoredPoint(id=457, version=3, score=0.123575434, payload={'artist': 'Anna Johnson', 'country': 'Australia', 'song': 'hospital behavior success', 'url_song': 'http://santiago.org/', 'year': '1980'}, vector=None),
 ScoredPoint(id=170, version=3, score=-0.076150686, payload={'artist': 'Benjamin Rogers', 'country': 'Australia', 'song': 'person big quality', 'url_song': 'http://www.gilmore-munoz.com/', 'year': '1993'}, vector=None)]

Lastly, say we want aussie songs but we don't care how new or old these songs are. Let's exclude points based on the year contained in the payload.

In [27]:
client.search(
    collection_name=my_collection,
    query_vector=living_la_vida_loca,
    query_filter=aussie_songs,
    with_payload=models.PayloadSelectorExclude(exclude=["year"]),
    limit=5
)

[ScoredPoint(id=457, version=3, score=0.123575434, payload={'artist': 'Anna Johnson', 'country': 'Australia', 'song': 'hospital behavior success', 'url_song': 'http://santiago.org/'}, vector=None),
 ScoredPoint(id=170, version=3, score=-0.076150686, payload={'artist': 'Benjamin Rogers', 'country': 'Australia', 'song': 'person big quality', 'url_song': 'http://www.gilmore-munoz.com/'}, vector=None),
 ScoredPoint(id=542, version=3, score=-0.07928085, payload={'artist': 'Charles Pennington', 'country': 'Australia', 'song': 'car help throughout', 'url_song': 'https://www.owens.com/'}, vector=None),
 ScoredPoint(id=115, version=3, score=-0.27341512, payload={'artist': 'Jane Simmons', 'country': 'Australia', 'song': 'full so analysis', 'url_song': 'https://skinner-maynard.com/'}, vector=None)]

As you can see, you can apply a wide-range of filtering methods to allows your users to take more control of the recommendations they are being served.

If you wanted to clear out the payload and upload a new for the same vectors, you can use `client.clear_payload()` as below.

```python
client.clear_payload(
    collection_name=my_collection,
    points_selector=models.PointIdsList(
        points=index,
    )
)
```

## 5. Recommendations

A recommendation system is a technology that suggests items or content to users based on their preferences, interests, or past behavior. It's like having a knowledgeable friend who can recommend movies, books, music, or products that you might enjoy.

In its most widely-used form, recommendation systems work by analyzing a large amount of data about you and other users. The system looks at your previous choices, such as movies you've watched, products you've bought, or articles you've read, and it then compares this information with data from other people who have similar tastes or interests. By finding patterns and similarities, the system can predict what you might like and show it to you.

Recommendation systems are used in various online platforms, such as streaming services like Netflix, e-commerce websites like Amazon, social media platforms like Facebook, and even music apps like Spotify. They aim to personalize your experience, save you time searching for things you might like, or introduce you to new and relevant content that you may not have discovered otherwise.

In a nutshell, a recommendation system is a smart tool that helps you discover new things you'll probably enjoy based on your preferences and the experiences of others.

Qdrant offers a convenient API that allows you to take into account user feedback by including similar songs to those the user have already liked (👍), or, conversely, by excluding songs that are similar to those the user has signal it did not like (👎).

The method is straightforward to implement via the `client.recommend()` method, and it provides enough flexibility that the logic on how the feedback gets capture can rest in the hands of the developers of your organization's platform. So, what do you need to keep in mind when making recommendations with Qdrant?

- `collection_name=` - from which collection are we selecting vectors.
- `query_filter=` - which filter will we apply to our search, if any.
- `negative=` - are there any songs the user explicitly didn't like? If so, let's use the `id` of these songs to exclude semantically similar ones.
- `positive=` - are there any songs the user explicitly liked? If so, let's use the `id` of these songs to include semantically similar ones.
- `limit=` - how many songs should we show our user.

One last to note is that the `positive=` parameter is a required one but the negative one isn't/

With this new knowledge under our sleeves, imagine there are two songs, "[Suegra](https://www.youtube.com/watch?v=p7ff5EntWsE&ab_channel=RomeoSantosVEVO)" by Romeo Santos and is "[Worst Behavior](https://www.youtube.com/watch?v=U5pzmGX8Ztg&ab_channel=DrakeVEVO)" by Drake, represented by the ids 17 and 120 respectively.

In [28]:
client.recommend(
    collection_name=my_collection,
    query_vector=living_la_vida_loca,
    positive=[17],
    limit=5
)

[ScoredPoint(id=872, version=3, score=0.36140996, payload={'artist': 'Robert Keith', 'country': 'Bangladesh', 'song': 'would audience recently', 'url_song': 'https://www.schmidt.org/', 'year': '2012'}, vector=None),
 ScoredPoint(id=880, version=3, score=0.32057765, payload={'artist': 'Adam Mason', 'country': 'Congo', 'song': 'nor collection response', 'url_song': 'http://goodwin.com/', 'year': '2003'}, vector=None),
 ScoredPoint(id=762, version=3, score=0.30006018, payload={'artist': 'Raymond Fisher', 'country': 'Guadeloupe', 'song': 'their region party', 'url_song': 'http://www.clark-jenkins.com/', 'year': '2009'}, vector=None),
 ScoredPoint(id=637, version=3, score=0.27169442, payload={'artist': 'Michael Johnson', 'country': 'Rwanda', 'song': 'no third clearly', 'url_song': 'http://flores.com/', 'year': '2010'}, vector=None),
 ScoredPoint(id=328, version=3, score=0.2697419, payload={'artist': 'Jason Haynes', 'country': 'Guadeloupe', 'song': 'become else toward', 'url_song': 'http://w

In [29]:
client.recommend(
    collection_name=my_collection,
    query_vector=living_la_vida_loca,
    positive=[17],
    negative=[120],
    limit=5
)

[ScoredPoint(id=762, version=3, score=0.32661206, payload={'artist': 'Raymond Fisher', 'country': 'Guadeloupe', 'song': 'their region party', 'url_song': 'http://www.clark-jenkins.com/', 'year': '2009'}, vector=None),
 ScoredPoint(id=437, version=3, score=0.2956204, payload={'artist': 'Brian Reed', 'country': 'American Samoa', 'song': 'newspaper form determine', 'url_song': 'http://stewart-deleon.org/', 'year': '2022'}, vector=None),
 ScoredPoint(id=784, version=3, score=0.29439625, payload={'artist': 'Abigail Price', 'country': 'Madagascar', 'song': 'question inside really', 'url_song': 'http://www.lewis-pratt.com/', 'year': '2012'}, vector=None),
 ScoredPoint(id=872, version=3, score=0.28801492, payload={'artist': 'Robert Keith', 'country': 'Bangladesh', 'song': 'would audience recently', 'url_song': 'https://www.schmidt.org/', 'year': '2012'}, vector=None),
 ScoredPoint(id=880, version=3, score=0.2876037, payload={'artist': 'Adam Mason', 'country': 'Congo', 'song': 'nor collection r

Notice that, while the similarity scores are completely random, it is important that we pay attention to these when serving recommendations in production. Even if we get 5 vectors back, it might more useful to show random results rather than vectors that are 0.012 similar to the query vector. With this in mind, we can actually set a threshold for our vectors with the `score_threshold=` parameter.

In [30]:
client.recommend(
    collection_name=my_collection,
    query_vector=living_la_vida_loca,
    positive=[17],
    negative=[120, 180],
    score_threshold=0.22,
    limit=5
)

[ScoredPoint(id=872, version=3, score=0.3163067, payload={'artist': 'Robert Keith', 'country': 'Bangladesh', 'song': 'would audience recently', 'url_song': 'https://www.schmidt.org/', 'year': '2012'}, vector=None),
 ScoredPoint(id=553, version=3, score=0.3123151, payload={'artist': 'Gina Taylor', 'country': 'Liberia', 'song': 'member manage arrive', 'url_song': 'https://foster.com/', 'year': '2002'}, vector=None),
 ScoredPoint(id=880, version=3, score=0.3113261, payload={'artist': 'Adam Mason', 'country': 'Congo', 'song': 'nor collection response', 'url_song': 'http://goodwin.com/', 'year': '2003'}, vector=None),
 ScoredPoint(id=437, version=3, score=0.29122013, payload={'artist': 'Brian Reed', 'country': 'American Samoa', 'song': 'newspaper form determine', 'url_song': 'http://stewart-deleon.org/', 'year': '2022'}, vector=None),
 ScoredPoint(id=762, version=3, score=0.27905738, payload={'artist': 'Raymond Fisher', 'country': 'Guadeloupe', 'song': 'their region party', 'url_song': 'htt

Lastly, we can add filters in the same way as we did before. Note that these filters could be tags that your users pick like genres including `reggeaton`, `bachata`, and `salsa` (sorry Drake), or the language of the song.

In [31]:
client.recommend(
    collection_name=my_collection,
    query_vector=living_la_vida_loca,
    query_filter=models.Filter(
        must=[models.FieldCondition(key="country", match=models.MatchValue(value="Dominican Republic"))]
    ),
    positive=[17],
    negative=[120],
    limit=5
)

[ScoredPoint(id=614, version=3, score=0.1519368, payload={'artist': 'Matthew Meyer', 'country': 'Dominican Republic', 'song': 'address food former', 'url_song': 'https://www.ford.org/', 'year': '1992'}, vector=None),
 ScoredPoint(id=874, version=3, score=0.12257638, payload={'artist': 'David Martinez', 'country': 'Dominican Republic', 'song': 'trip physical while', 'url_song': 'https://kim-ward.com/', 'year': '1980'}, vector=None),
 ScoredPoint(id=602, version=3, score=0.10234953, payload={'artist': 'Brenda Guerrero', 'country': 'Dominican Republic', 'song': 'might be debate', 'url_song': 'http://www.dennis.net/', 'year': '1981'}, vector=None),
 ScoredPoint(id=269, version=3, score=0.052880105, payload={'artist': 'Timothy Brown', 'country': 'Dominican Republic', 'song': 'current sister hot', 'url_song': 'http://campos.biz/', 'year': '1986'}, vector=None),
 ScoredPoint(id=23, version=3, score=0.044240832, payload={'artist': 'Joshua Adams', 'country': 'Dominican Republic', 'song': 'that 

That's it! You have now gone over a whirlwind tour of vector databases and are ready to tackle new challenges. 😎

## 5. Conclusion

To wrap up, we have explored a bit of the fascinating world of vector databases, and learned that these databases provide efficient storage and retrieval of high-dimensional vectors, making them ideal for similarity-based search tasks and recommendation systems. Both of these use cases can be applied in a variety of industries while helping us unlock new levels of information retrieval.

## 6. Resources

Here is a list with some resources that we found useful, and that helped with the development of this tutorial.

- [Fine Tuning Similar Cars Search](https://qdrant.tech/articles/cars-recognition/)
- [Q&A with Similarity Learning](https://qdrant.tech/articles/faq-question-answering/)
- [Question Answering with LangChain and Qdrant without boilerplate](https://qdrant.tech/articles/langchain-integration/)
- [Extending ChatGPT with a Qdrant-based knowledge base](https://qdrant.tech/articles/chatgpt-plugin/)
- [Word Embedding and Word2Vec, Clearly Explained!!!](https://www.youtube.com/watch?v=viZrOnJclY0&ab_channel=StatQuestwithJoshStarmer) by StatQuest with Josh Starmer
- [Word Embeddings, Bias in ML, Why You Don't Like Math, & Why AI Needs You](https://www.youtube.com/watch?v=25nC0n9ERq4&ab_channel=RachelThomas) by Rachel Thomas